In [1]:
pip install pyzipper

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
python--version

In [1]:
pip install pyzipper
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import zipfile
import os
import json
import pyzipper  # for password-protected zipping

# Initialize main window
root = tk.Tk()
root.title("Enhanced File Zipper")
root.config(bg="pink")

# Variables
file_list = []
compression_var = tk.StringVar(value="Deflated (Standard)")  # Corrected initialization

# Functions
def select_files():
    files = filedialog.askopenfilenames(title="Select Files to Zip")
    file_list.extend(files)
    update_file_list()

def zip_files():
    if not file_list:
        messagebox.showwarning("Warning", "No files selected!")
        return

    zip_name = filedialog.asksaveasfilename(defaultextension=".zip", title="Save Zip File As")
    if not zip_name:
        return  # User cancelled the save dialog

    password = password_entry.get().encode('utf-8') if password_entry.get() else None
    compression_type = compression_options[compression_var.get()]
    progress['maximum'] = len(file_list)

    # Calculate original size
    original_size = sum(os.path.getsize(f) for f in file_list)

    try:
        # Compress with or without password
        if password:
            with pyzipper.AESZipFile(zip_name, 'w', compression=compression_type, encryption=pyzipper.WZ_AES) as zipf:
                zipf.setpassword(password)
                for i, file in enumerate(file_list):
                    zipf.write(file, os.path.basename(file))
                    progress['value'] = i + 1
                    root.update_idletasks()
        else:
            with zipfile.ZipFile(zip_name, 'w', compression=compression_type) as zipf:
                for i, file in enumerate(file_list):
                    zipf.write(file, os.path.basename(file))
                    progress['value'] = i + 1
                    root.update_idletasks()

        # Calculate compressed size and compression ratio
        compressed_size = os.path.getsize(zip_name)
        ratio = (1 - compressed_size / original_size) * 100
        messagebox.showinfo("Compression Stats", f"Original Size: {original_size/1024:.2f} KB\n"
                                                 f"Compressed Size: {compressed_size/1024:.2f} KB\n"
                                                 f"Compression Ratio: {ratio:.2f}%")
        
        # Save recent file to history
        save_recent(zip_name)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

def unzip_file():
    zip_file = filedialog.askopenfilename(title="Select Zip File to Unzip", filetypes=[("Zip files", "*.zip")])
    unzip_folder = filedialog.askdirectory(title="Select Folder to Extract Files To")
    
    if not zip_file or not unzip_folder:
        return  # User cancelled the dialog

    password = password_entry.get().encode('utf-8') if password_entry.get() else None

    try:
        with pyzipper.AESZipFile(zip_file, 'r') as zipf:
            if password:
                zipf.pwd = password
            zipf.extractall(unzip_folder)
        messagebox.showinfo("Success", f"Files extracted to {unzip_folder}")
        save_recent(zip_file)
    except RuntimeError:
        messagebox.showerror("Error", "Password required or incorrect password.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while unzipping: {str(e)}")


def update_file_list():
    file_display.delete(0, tk.END)
    for file in file_list:
        file_display.insert(tk.END, file)

def save_recent(zip_name):
    recent_files = []
    try:
        with open('recent_files.json', 'r') as f:
            recent_files = json.load(f)
    except FileNotFoundError:
        pass
    recent_files.append(zip_name)
    with open('recent_files.json', 'w') as f:
        json.dump(recent_files[-5:], f)  # Store only the 5 most recent files

def show_recent():
    file_display.delete(0, tk.END)
    try:
        with open('recent_files.json', 'r') as f:
            recent_files = json.load(f)
        for file in recent_files:
            file_display.insert(tk.END, file)
    except FileNotFoundError:
        file_display.insert(tk.END, "No recent files.")

# Widgets
tk.Button(root, text="Select Files", command=select_files).pack()
file_display = tk.Listbox(root, width=50, height=10)
file_display.pack()

# Compression Level Selection
compression_options = {"No Compression": zipfile.ZIP_STORED, "Deflated (Standard)": zipfile.ZIP_DEFLATED}
tk.Label(root, text="Compression Level").pack()
compression_menu = tk.OptionMenu(root, compression_var, *compression_options.keys())
compression_menu.pack()

# Password Entry
tk.Label(root, text="Password (Optional)").pack()
password_entry = tk.Entry(root, show='*')
password_entry.pack()

# Progress Bar
progress = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
progress.pack()

# Action Buttons
tk.Button(root, text="Zip Files", command=zip_files).pack()
tk.Button(root, text="Unzip File", command=unzip_file).pack()
tk.Button(root, text="Show Recent Files", command=show_recent).pack()

# Run main loop
root.mainloop()
